In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, sum, round

spark = SparkSession.builder.appName("analysefootball").getOrCreate()
print(" la session a été creer avec succès")

 la session a été creer avec succès


In [2]:
df_matches = spark.read.format('csv').options(header = 'True').load('data/football_matches.csv')
df_matches.limit(10).show()

+--------+---+------+----------+--------------+--------------+----+----+---+
|Match_ID|Div|Season|      Date|      HomeTeam|      AwayTeam|FTHG|FTAG|FTR|
+--------+---+------+----------+--------------+--------------+----+----+---+
|       1| D2|  2009|2010-04-04|    Oberhausen|Kaiserslautern|   2|   1|  H|
|       2| D2|  2009|2009-11-01|   Munich 1860|Kaiserslautern|   0|   1|  A|
|       3| D2|  2009|2009-10-04| Frankfurt FSV|Kaiserslautern|   1|   1|  D|
|       4| D2|  2009|2010-02-21| Frankfurt FSV|     Karlsruhe|   2|   1|  H|
|       5| D2|  2009|2009-12-06|         Ahlen|     Karlsruhe|   1|   3|  A|
|       6| D2|  2009|2010-04-03|  Union Berlin|     Karlsruhe|   1|   1|  D|
|       7| D2|  2009|2009-08-14|     Paderborn|     Karlsruhe|   2|   0|  H|
|       8| D2|  2009|2010-03-08|     Bielefeld|     Karlsruhe|   0|   1|  A|
|       9| D2|  2009|2009-09-26|Kaiserslautern|     Karlsruhe|   2|   0|  H|
|      10| D2|  2009|2009-11-21| Hansa Rostock|     Karlsruhe|   2|   1|  H|

In [3]:
# renomer les colones principales
df_matches = df_matches.selectExpr(
    "*",
    "`FTHG` AS `ButsEquipeDomicile`",
    "`FTAG` AS `ButsEquipeExterieur`",
    "`FTR` AS `ResultatFinal`",
)
df_matches = df_matches.drop("FTHG","FTAG","FTR")

#  creation des colonnes binaires pour victoire, defaite et matche null
df_matches = (df_matches.withColumn("VictoireDomicile", when(col("ResultatFinal") == "H", 1).otherwise(0)).withColumn("VictoireExterieur", when(col("ResultatFinal") == "A", 1).otherwise(0)).withColumn("MatchNull", when(col("ResultatFinal") == "D", 1).otherwise(0)))

df_matches.limit(10).show()

+--------+---+------+----------+--------------+--------------+------------------+-------------------+-------------+----------------+-----------------+---------+
|Match_ID|Div|Season|      Date|      HomeTeam|      AwayTeam|ButsEquipeDomicile|ButsEquipeExterieur|ResultatFinal|VictoireDomicile|VictoireExterieur|MatchNull|
+--------+---+------+----------+--------------+--------------+------------------+-------------------+-------------+----------------+-----------------+---------+
|       1| D2|  2009|2010-04-04|    Oberhausen|Kaiserslautern|                 2|                  1|            H|               1|                0|        0|
|       2| D2|  2009|2009-11-01|   Munich 1860|Kaiserslautern|                 0|                  1|            A|               0|                1|        0|
|       3| D2|  2009|2009-10-04| Frankfurt FSV|Kaiserslautern|                 1|                  1|            D|               0|                0|        1|
|       4| D2|  2009|2010-02-21| F

In [15]:
# filtre D1 entre 2000 et 2015
df_bundesliga = df_matches.filter((col("Div") == "D1") & (col("Season") >= 2000 ) & (col("Season") <= 2015 ))
df_bundesliga.limit(10).show()

+--------+---+------+----------+-------------+----------+------------------+-------------------+-------------+----------------+-----------------+---------+
|Match_ID|Div|Season|      Date|     HomeTeam|  AwayTeam|ButsEquipeDomicile|ButsEquipeExterieur|ResultatFinal|VictoireDomicile|VictoireExterieur|MatchNull|
+--------+---+------+----------+-------------+----------+------------------+-------------------+-------------+----------------+-----------------+---------+
|      21| D1|  2009|2010-02-06|       Bochum|Leverkusen|                 1|                  1|            D|               0|                0|        1|
|      22| D1|  2009|2009-11-22|Bayern Munich|Leverkusen|                 1|                  1|            D|               0|                0|        1|
|      23| D1|  2009|2010-05-08|   M'gladbach|Leverkusen|                 1|                  1|            D|               0|                0|        1|
|      24| D1|  2009|2009-08-08|        Mainz|Leverkusen|       

In [ ]:
# statiqtique des matchs à domicile 
df_domicile = df_bundesliga.groupBy('Season', 'HomeTeam') \
    .agg(
        sum('VictoireDomicile').alias('TotalVictoireDomicile'),
        sum('VictoireExterieur').alias('TotaldefaitesDomicile'),
        sum('MatchNull').alias('TotalMatchNullDomicile'),
        sum('ButsEquipeDomicile').alias('TotalButsMarquesDomicile'),
        sum('ButsEquipeExterieur').alias('TotalButsEcaissesDomicile')
    ) \
    .withColumnRenamed('HomeTeam', 'Equipe')

df_domicile.limit(10).show()

+------+--------------+---------------------+---------------------+----------------------+------------------------+--------------------+
|Season|        Equipe|TotalVictoireDomicile|TotaldefaitesDomicile|TotalMatchNullDomicile|TotalButsMarquesDomicile|ButsEcaissesDomicile|
+------+--------------+---------------------+---------------------+----------------------+------------------------+--------------------+
|  2011|       Hamburg|                    3|                    7|                     7|                    19.0|                29.0|
|  2005|Kaiserslautern|                    5|                    7|                     5|                    26.0|                33.0|
|  2006|       Cottbus|                    6|                    6|                     5|                    21.0|                22.0|
|  2001|      St Pauli|                    4|                    9|                     4|                    19.0|                28.0|
|  2005|         Mainz|                  

In [19]:
# statiqtique des matchs à l'exterieur 
df_exterieur = df_bundesliga.groupBy('Season', 'AwayTeam') \
    .agg(
        sum('VictoireExterieur').alias('TotalVictoireExterieur'),
        sum('VictoireDomicile').alias('TotaldefaitesExterieur'),
        sum('MatchNull').alias('TotalMatchNullExterieur'),
        sum('ButsEquipeExterieur').alias('TotalButsMarquesExterieur'),
        sum('ButsEquipeDomicile').alias('TotalButsEncaissesExterieur'),
    ) \
    .withColumnRenamed('AwayTeam', 'Equipe')

df_domicile.limit(10).show()

+------+--------------+---------------------+---------------------+----------------------+------------------------+--------------------+
|Season|        Equipe|TotalVictoireDomicile|TotaldefaitesDomicile|TotalMatchNullDomicile|TotalButsMarquesDomicile|ButsEcaissesDomicile|
+------+--------------+---------------------+---------------------+----------------------+------------------------+--------------------+
|  2011|       Hamburg|                    3|                    7|                     7|                    19.0|                29.0|
|  2005|Kaiserslautern|                    5|                    7|                     5|                    26.0|                33.0|
|  2006|       Cottbus|                    6|                    6|                     5|                    21.0|                22.0|
|  2001|      St Pauli|                    4|                    9|                     4|                    19.0|                28.0|
|  2005|         Mainz|                  